# Recognizing hand-written digits: checking the network

In [ ]:
from sklearn import datasets, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

import matplotlib.pyplot as plt
%matplotlib inline

## The digits dataset

The data that we are interested in is a set of 1797 images of digits, each one made of $8\times8$ pixels.

In [ ]:
digits = datasets.load_digits()

To apply a classifier on this data, we need to flatten the image, to
turn the data in a (samples, feature) matrix:

In [ ]:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

## Cross-validation: evaluating network performance

Learning the parameters of the MLP and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called [**overfitting**](https://en.wikipedia.org/wiki/Overfitting). To avoid it, it is common practice  to hold out part of the available data as a test set `[X_test, y_test]`. 

In scikit-learn a random split into training and test sets can be quickly computed with the [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) helper function.

In [ ]:
X, y = data, digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

## Data scaling

Standardization of datasets is a common requirement for neural networks; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance.

In scikit-learn, the preprocessing module provides a utility class [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) that computes the mean and standard deviation on a training set so as to be able to later reapply the same transformation on the testing set.

In [ ]:
# Don't cheat - load the scaler from file
scaler = joblib.load('digits_scaler.pkl') 
X_train = scaler.transform(X_train)
# apply same transformation to test data
X_test = scaler.transform(X_test)

## Load the network

In [ ]:
net = joblib.load('digits_net.pkl')

## Analysis of the network

In scikit-learn there are some functions for measuring the accuracy of the classification.

### Classification report

The [classification_report](http://scikit-learn.org/stable/modules/model_evaluation.html#classification-report) function builds a text report showing the main classification metrics:

* [precision](https://en.wikipedia.org/wiki/Precision_and_recall): ratio $\frac{tp}{tp + fp}$ where $tp$ is the number of true positives and $fp$ the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
* [recall](https://en.wikipedia.org/wiki/Precision_and_recall): ratio $\frac{tp}{tp + fn}$ where $tp$ is the number of true positives and $fn$ the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
* [f1-score](https://en.wikipedia.org/wiki/F1_score): a weighted average of the precision and recall, where an $F_1$ score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the $F_1$ score are equal. The formula for the $F_1$ score is: $F_1 = 2\cdot\frac{precision \cdot recall}{precision + recall}$
* support: the support is the number of occurrences of each class

In [ ]:
expected = y_test
predicted = net.predict(X_test)
print(metrics.classification_report(expected, predicted))

### Confusion matrix

The [`confusion_matrix`](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function evaluates classification accuracy by computing the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix).

By definition, a confusion matrix $C$ is such that $C_{i, j}$ is equal to the number of observations known to be in group $i$ but predicted to be in group $j$. Thus an optimal classification result would produce a diagonal matrix.

In [ ]:
print(metrics.confusion_matrix(expected, predicted))

### Training curve

Number of iterations during training:

In [ ]:
net.n_iter_

Loss curve: (currently, [MLPClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier) supports only the [Cross-Entropy loss function](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression))

In [ ]:
plt.plot(net.loss_curve_);
plt.xlabel('Iterations');
plt.ylabel('Loss');